# 9. Statics: Class-Level Variables and Methods

While object attributes (`self.attribute`) store data for each individual instance, sometimes we need data and functions that belong to the **class blueprint itself**. These are "static" members, meaning they are shared across all instances of the class.

- **Class Variables (or Static Variables):** Attributes shared by all instances of a class.
- **Static Methods:** Utility functions grouped within a class that don't need instance data.
- **Class Methods:** Methods that operate on the class itself, not the instance.

## 9.1. Class Variables & Constants
Class variables are defined directly inside the class, but outside of any method.
They are **identical** and **shared** by all instances created from that class.
- They are ideal for storing constants or data that is common to all objects of that type.
- It is best practice to access them via the class name (e.g., `ClassName.variable`).


In [ ]:
class MarsData: # A class containing only class variables, like a data record
    # Constants are, by convention, written in ALL_CAPS.
    # They are not meant to be changed.
    GRAVITY = 3.711 # m/s^2
    PLANET_TYPE = "Terrestrial"
    AVG_TEMP_CELSIUS = -63

# Accessing class variables directly through the class name
print(MarsData.GRAVITY) # -> 3.711

class TerraformedPlanet:
    base_atmosphere_pressure = 1.0 # A class variable 
    HAS_LIQUID_WATER = True # A class constant

    def __init__(self, name: str, population: int):
        # Instance variables (attributes) are unique to each object
        self.name = name
        self.population = population


# --- Testing ---
# Accessing class variables via the class
print(TerraformedPlanet.base_atmosphere_pressure)

colony_one = TerraformedPlanet("New Hope", 50000)
# You can also access class variables via an instance
print(colony_one.HAS_LIQUID_WATER)
print(colony_one.base_atmosphere_pressure) # Looks up on the class


class TerraformingProject:
    # Class variables used as a counter and a registry for all projects
    project_count = 0
    minimum_budget = 1_000_000 # Using underscore for readability
    active_projects_log = [] # A shared list for all instances

    def __init__(self, project_name: str, target_planet: str):
        self.name = project_name # Instance attribute
        self.planet = target_planet # Instance attribute

        # Accessing class variables via the ClassName to modify them
        TerraformingProject.project_count += 1
        TerraformingProject.active_projects_log.append(self.name)

# --- Testing ---
project_one = TerraformingProject("Mars 2045", "Mars")
project_two = TerraformingProject("Venus 2078", "Venus")
print(TerraformingProject.project_count)
print(TerraformingProject.active_projects_log)

## 9.2. Static Methods
A static method is a function defined inside a class that does not need access to instance data (`self`) or class data (`cls`). It's essentially a **utility function grouped with a class** for logical organization.
- It is essentially a utility function that is grouped with a class for logical organization.
- Its behavior is always the same, regardless of the class or instance it's called on.
- It is defined with the `@staticmethod` decorator.
- It can be called on the class or an instance of the class.

In [ ]:
class PlanetaryPhysics:

    @staticmethod # This method has no 'self'
    def calculate_escape_velocity(mass_kg: float, radius_km: float) -> float:
        # A general physics calculation related to planets, but not to a specific one
        G = 6.67430e-11 # Gravitational constant
        mass = mass_kg
        radius_m = radius_km * 1000
        velocity = (2 * G * mass / radius_m) ** 0.5
        return velocity
        
    @staticmethod
    def is_in_habitable_zone(distance_from_star_au: float) -> bool:
        # A general check, not tied to a specific object's state
        return 0.95 < distance_from_star_au < 1.68


# --- Testing ---
# You can call a static method via the CLASS...
escape_v_earth = PlanetaryPhysics.calculate_escape_velocity(5.972e24, 6371)
print(f"Earth's approximate escape velocity: {escape_v_earth:.2f} m/s")

# ...or via an INSTANCE (though less common)
physics_calculator = PlanetaryPhysics()
print(f"Is Earth in habitable zone? {physics_calculator.is_in_habitable_zone(1.0)}")

## 9.3. Class Methods
A class method is a method that operates on the **class itself**, not the instance.
- Its first parameter is always the class (`cls`), passed automatically by Python.
- It **can access and modify class variables**, affecting all instances.
- It cannot access instance variables (`self.attribute`).
- It is often used for creating "factory methods" – alternative constructors.
- It is defined with the `@classmethod` decorator.

In [ ]:
""" Example 1: Managing Class-Level State """

class ProjectRegistry:
    active_project_count = 0 # Class variable, shared by all instances

    def __init__(self):
        ProjectRegistry.active_project_count += 1 # Increment the class counter for each new instance

    @classmethod
    def get_active_projects(cls): # This method receives the class 'ProjectRegistry' as 'cls'
        return cls.active_project_count

    @classmethod
    def reset_counter(cls): # This method modifies the class variable
        cls.active_project_count = 0


# --- Testing ---
print(f"Initial project count: {ProjectRegistry.get_active_projects()}")
p1 = ProjectRegistry()
p2 = ProjectRegistry()

print(f"Count after creating instances: {ProjectRegistry.get_active_projects()}")
ProjectRegistry.reset_counter() # Call class method on the class
print(f"Count after reset: {ProjectRegistry.get_active_projects()}")


""" Example 2: Factory Pattern """

# Using class methods as alternative constructors.
class TerraformingUnit:
    def __init__(self, unit_name: str, purpose: str):
        self.name = unit_name
        self.purpose = purpose
        print(f"Unit '{self.name}' of type '{self.purpose}' deployed.")

    @classmethod
    def create_atmospheric_processor(cls, name: str):
        # This factory method creates a specific type of unit
        return cls(name, purpose="Atmosphere Generation")

    @classmethod
    def create_hydrosphere_generator(cls, name: str):
        # This factory method creates another specific type
        return cls(name, purpose="Hydrosphere Seeding")


# --- Testing ---
# Instead of using the standard constructor, we use our factory methods
atmo_gen_1 = TerraformingUnit.create_atmospheric_processor("Atmo-Gen-1")
hydro_gen_1 = TerraformingUnit.create_hydrosphere_generator("Hydro-Gen-1")

# Use the standard constructor is still possible but that defies the purpose of the factory method
atmo_gen_2 = TerraformingUnit("Dummy", "Not specific")


""" Statics and Inheritance """

class MissionBriefing: # Parent class
    motto = "Explore strange new worlds!" # A class variable
    
    @classmethod
    def print_motto(cls):
        print(cls.motto) # Prints the 'motto' from the class it is called on

    @staticmethod
    def print_universal_directive():
        print("Directive 001: Do no harm.") # Static method behavior is fixed

class FirstContactBriefing(MissionBriefing): # Child class
    # Override the class variable
    motto = "Seek out new life and new civilizations."


# --- Testing ---
# Call methods on parent class instance
base_briefing = MissionBriefing()
base_briefing.print_universal_directive() # -> "Directive 001: Do no harm."
base_briefing.print_motto() # -> "Explore strange new worlds!"

# Call methods on child class instance
contact_briefing = FirstContactBriefing()
contact_briefing.print_universal_directive() # -> "Directive 001: Do no harm." (staticmethod is unchanged)
contact_briefing.print_motto() # -> "Seek out new life..." 

## Recommendation on Statics:
- Call class methods on the CLASS, not the instance, for clarity!
- A `@staticmethod` is the same in both parent and child.
- Choose `@classmethod` if you plan to use inheritance and want the method to work with the subclass.
- While decorators are not strictly mandatory for creating static-like methods, using them is best practice for readability, clarity, and enabling more advanced programming patterns.

## practice

**Task: Autonomous Drone Fleet Management**
- **Scenario:** You are tasked with managing the production and registry of a fleet of autonomous drones for various missions.
- **Requirements:**
    - Create a class named `AutonomousDrone`. It should have instance attributes for `drone_id`, `model`, and `max_speed`.
    - Keep a running count of the total number of drones manufactured. The count should increment each time a new instance is created.
    - Create a **class method** that returns this total count.
- **Testing:**
    - Create 3 different drone instances.
    - Call your class method to print the total count and verify it is 3.

---
**Challenge I: Instance Registry & Reporting**
- **Requirements:**
    - Extend your `AutonomousDrone` class.
    - Add another **class variable** to act as a registry that stores every drone instance upon creation.
    - When you use a print() on an class instance return a user-friendly string summary of a drone's details (e.g., `"Drone [ID: X-01, Model: Scout, Speed: 120 kph]"`).
    - Create a **class method** goes through all manufactured drones and prints the details of only those whose `max_speed` is greater than the `speed_limit`.

---
**Challenge II: Drone Factory**
- **Requirements:**
    - Refactor your `AutonomousDrone` class to act as a "factory" for standard models.
    - Using **class methods** as alternative constructors, create two factory methods:
        - `create_scout_drone`: This method should create and return an `AutonomousDrone` instance with pre-filled parameters: `model="ReconScout"` and `max_speed=120`.
        - `create_cargo_drone`: This method should create and return an `AutonomousDrone` instance with pre-filled parameters: `model="CargoLifter"` and `max_speed=75`.
- **Testing:**
    - Use your new factory methods to create one scout drone and one cargo drone.
    - Print the objects to verify their attributes.

---
#### © Jiří Svoboda (George Freedom)
- Web: https://GeorgeFreedom.com
- LinkedIn: https://www.linkedin.com/in/georgefreedom/
- Book me: https://cal.com/georgefreedom